In [51]:
from grizly import set_cwd, QFrame, join, union

sq_json = set_cwd("acoe_projects", "blank_project", "blank_subquery.json")

sq_forecast = QFrame(engine="engine_str").read_json(json_path=sq_json, subquery="sales_forecast")
sq_flash = QFrame(engine="engine_str").read_json(json_path=sq_json, subquery="sales_flash")

In [53]:
sq_forecast.engine

'engine_str'

In [28]:
on_join = "sq1.grouped_dimensions = sq2.grouped_dimensions"
joined = join(qframes=[sq_forecast, sq_flash], join_type="INNER JOIN", on = on_join)

Data joined successfully.


In [29]:
print(joined.get_sql().sql)

SELECT sq1.dimension_new_name AS dimension_new_name,
       sq1.sales_value AS sales_value,
       sq2.another_grouped_dim AS another_grouped_dim
FROM
  (SELECT grouped_forecast_dimension AS dimension_new_name,
          sum(grouped_forecast_value) AS sales_value
   FROM forecast_schema.forecast_table
   GROUP BY dimension_new_name) sq1
INNER JOIN
  (SELECT grouped_dimension AS dimension_new_name,
          another_grouped_dim,
          sum(grouped_value) AS sales_value
   FROM example_schema.example_table
   GROUP BY dimension_new_name,
            another_grouped_dim) sq2 ON sq1.grouped_dimensions = sq2.grouped_dimensions


In [40]:
sq_forecast.get_fields()

['grouped_forecast_dimension', 'grouped_forecast_value']

In [41]:
joined.get_fields()

['sq1.dimension_new_name', 'sq1.sales_value', 'sq2.another_grouped_dim']

In [46]:
sq_forecast.select(["grouped_forecast_dimension"])
joined.select(["sq1.dimension_new_name"])

Field sq1.dimension_new_name not found


In [47]:
unioned = union(qframes=[sq_forecast, joined], union_type="UNION ALL")

Data unioned successfully.


In [48]:
print(unioned.get_sql().sql)

SELECT sq.dimension_new_name AS dimension_new_name
FROM
  (SELECT grouped_forecast_dimension AS dimension_new_name,
          sum(grouped_forecast_value) AS sales_value
   FROM forecast_schema.forecast_table
   GROUP BY dimension_new_name) sq
UNION ALL
SELECT sq.dimension_new_name AS dimension_new_name
FROM
  (SELECT sq1.dimension_new_name AS dimension_new_name,
          sq1.sales_value AS sales_value,
          sq2.another_grouped_dim AS another_grouped_dim
   FROM
     (SELECT grouped_forecast_dimension AS dimension_new_name,
             sum(grouped_forecast_value) AS sales_value
      FROM forecast_schema.forecast_table
      GROUP BY dimension_new_name) sq1
   INNER JOIN
     (SELECT grouped_dimension AS dimension_new_name,
             another_grouped_dim,
             sum(grouped_value) AS sales_value
      FROM example_schema.example_table
      GROUP BY dimension_new_name,
               another_grouped_dim) sq2 ON sq1.grouped_dimensions = sq2.grouped_dimensions) sq


In [54]:
unioned.to_df()

InterfaceError: (pyodbc.InterfaceError) ('IM002', '[IM002] [Microsoft][ODBC Driver Manager] Data source name not found and no default driver specified (0) (SQLDriverConnect)')
(Background on this error at: http://sqlalche.me/e/rvf5)